In [44]:
import sys
import os
import os.path
from pathlib import Path

import numpy as np
import pandas as pd

import json
import re

import tkinter as tk
from tkinter import filedialog
from pathlib import Path


FOLDERS AND INPUT FILES

In [45]:
current_folder = Path(os.getcwd()).parents[0]
survey_folder = current_folder / 'data' / 'manual_survey'
print(survey_folder)

building_elements_survey = survey_folder / 'building_elements_survey.txt'
energy_elements_survey = survey_folder / 'energy_elements_survey.txt'

c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\manual_survey


BUILDING ELEMENTS

In [46]:
def sanitize_value(value):
    sanitized_value = re.sub(r'[^\d.-]', '', value)
    try:
        return float(sanitized_value) if sanitized_value else 0
    except ValueError:
        print(f"Error: Could not convert {value} to float.")
        return 0

def open_building_txt_file():
    filepath = filedialog.askopenfilename(
        title="Select the manual building elements survey file",
        filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
    )
    if not filepath:
        print("No file selected.")
        return None
    print(f"Selected file: {filepath}")
    return filepath


In [47]:
def compute_u_value_and_total_lambda(layers, Rint, Rext):
   
    if not layers:
        return None, None
    R_total = Rint + sum(thickness / lambda_layer for thickness, lambda_layer in layers) + Rext
    U_value = 1 / R_total
    total_thickness = sum(thickness for thickness, _ in layers)
    total_lambda = total_thickness / sum(thickness / lambda_layer for thickness, lambda_layer in layers)
    return U_value, total_lambda, total_thickness

def process_walls_file(txt_file_path, output_json_path):
    walls_data = []
    current_wall_type = None
    layers = []
    Rint = Rext = 0.0

    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip()
        if line.startswith("Rint"):
            Rint = sanitize_value(line.split('=')[1].strip())
        elif line.startswith("Rext"):
            Rext = sanitize_value(line.split('=')[1].strip())
        elif line.startswith("wall_type"):
            if current_wall_type:
                u_value, total_lambda, total_thickness = compute_u_value_and_total_lambda(layers, Rint, Rext)
                walls_data.append({
                    "wall_type": current_wall_type,
                    "total_thickness": total_thickness,
                    "layers": [{"thickness": thickness, "lambda": lambda_layer} for thickness, lambda_layer in layers],
                    "U_value": round(u_value, 2) if u_value is not None else "undefined",
                    "total_lambda": round(total_lambda, 2) if total_lambda is not None else "undefined"
                })
            current_wall_type = line.split('=')[1].strip()
            layers = []
        elif "thickness_layer" in line:
            thickness = sanitize_value(line.split('=')[1].strip())
        elif "lambda_layer" in line:
            lambda_value = sanitize_value(line.split('=')[1].strip())
            layers.append((thickness, lambda_value))

    if current_wall_type:
        u_value, total_lambda, total_thickness = compute_u_value_and_total_lambda(layers, Rint, Rext)
        walls_data.append({
            "wall_type": current_wall_type,
            "total_thickness": total_thickness,
            "layers": [{"thickness": thickness, "lambda": lambda_layer} for thickness, lambda_layer in layers],
            "U_value": round(u_value, 2) if u_value is not None else "undefined",
            "total_lambda": round(total_lambda, 2) if total_lambda is not None else "undefined"
        })

    with open(output_json_path, 'w') as json_file:
        json.dump({"walls_data": walls_data}, json_file, indent=4)
    print(f"Wall data successfully exported to {output_json_path}")

def process_windows_file(txt_file_path, output_json_path):
    windows_data = []
    current_window_type = None

    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip()
        if line.startswith("window_type"):
            current_window_type = line.split('=')[1].strip()
        elif "length_survey" in line:
            length_survey = sanitize_value(line.split('=')[1].strip())
        elif "height_survey" in line:
            height_survey = sanitize_value(line.split('=')[1].strip())
        elif "U_value" in line:
            u_value = sanitize_value(line.split('=')[1].strip())
            windows_data.append({
                "window_type": current_window_type,
                "length_survey": length_survey,
                "height_survey": height_survey,
                "U_value": u_value
            })

    with open(output_json_path, 'w') as json_file:
        json.dump({"windows_data": windows_data}, json_file, indent=4)
    print(f"Window data successfully exported to {output_json_path}")

def process_doors_file(txt_file_path, output_json_path):
    doors_data = []
    current_door_type = None

    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip()
        if line.startswith("door_type"):
            current_door_type = line.split('=')[1].strip()
        elif "length_survey" in line:
            length_survey = sanitize_value(line.split('=')[1].strip())
        elif "height_survey" in line:
            height_survey = sanitize_value(line.split('=')[1].strip())
        elif "U_value" in line:
            u_value = sanitize_value(line.split('=')[1].strip())
            doors_data.append({
                "door_type": current_door_type,
                "length_survey": length_survey,
                "height_survey": height_survey,
                "U_value": u_value
            })

    with open(output_json_path, 'w') as json_file:
        json.dump({"doors_data": doors_data}, json_file, indent=4)
    print(f"Door data successfully exported to {output_json_path}")



In [48]:
root = tk.Tk()
root.withdraw()

txt_file_path = open_building_txt_file()

if txt_file_path:
    output_json_walls_path = Path(os.getcwd()) / 'IDL_files' / 'IDL_walls_types.json'
    output_json_windows_path = Path(os.getcwd()) / 'IDL_files' / 'IDL_windows_types.json'
    output_json_doors_path = Path(os.getcwd()) / 'IDL_files' / 'IDL_doors_types.json'

    output_json_doors_path.parent.mkdir(exist_ok=True)

    process_walls_file(txt_file_path, output_json_walls_path)
    process_windows_file(txt_file_path, output_json_windows_path)
    process_doors_file(txt_file_path, output_json_doors_path)
else:
    print("No file selected. Exiting.")

Selected file: C:/Users/oscar/OneDrive - Fondazione Bruno Kessler/KUL_GeometricModel/data/manual_survey/building_elements_survey.txt
Wall data successfully exported to c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\IDL_walls_types.json
Window data successfully exported to c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\IDL_windows_types.json
Door data successfully exported to c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\IDL_doors_types.json


ENERGY ELEMENTS IN THE BUILDING

In [58]:
def open_energy_txt_file():

    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window
    file_path = filedialog.askopenfilename(
        title="Select the Energy Elements Survey File",
        filetypes=(("Text Files", "*.txt"), ("All Files", "*.*"))
    )

    return file_path if file_path else None


def sanitize_value(value):


    if re.match(r'^-?\d+(\.\d+)?$', value.strip()):
        return float(value) if '.' in value else int(value)
    else:
        return value.strip()


def parse_energy_system(txt_file_path):

    energy_data = {
        "title": "ENERGY SYSTEM OF THE BUILDING",
        "building_energy_system": {
            "heat_pump": [],
            "heating_system": {"types": []},
            "lighting_system": {"types": []},
            "hvac_system": [],
            "renewable_energy_sources": []
        }
    }

    current_section = None
    current_data = {}

    with open(txt_file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Identify the current section
            if "HEATING SYSTEM" in line:
                current_section = "heat_pump"
            elif "RADIATORS TYPE" in line:
                current_section = "heating_system"
            elif "LIGTH SYSTEM" in line:
                current_section = "lighting_system"
            elif "HVAC" in line:
                current_section = "hvac_system"
            elif "RENEWABLE ENERGY" in line:
                current_section = "renewable_energy_sources"

            # Parse key-value pairs in each line
            elif "=" in line:
                key, value = line.split('=', 1)
                key = key.strip().lower().replace(" ", "_")
                value = sanitize_value(value.strip())

                # Handle each section and structure data accordingly
                if current_section == "heat_pump":
                    if "building_energy_system" in key:
                        if current_data:
                            energy_data["building_energy_system"]["heat_pump"].append(current_data)
                        current_data = {}
                    current_data[key] = value

                elif current_section == "heating_system":
                    if "heating_system_type" in key:
                        if current_data:
                            energy_data["building_energy_system"]["heating_system"]["types"].append(current_data)
                        current_data = {}
                    current_data[key] = value

                elif current_section == "lighting_system":
                    if "lighting_system" in key:
                        if current_data:
                            energy_data["building_energy_system"]["lighting_system"]["types"].append(current_data)
                        current_data = {}
                    current_data[key] = value

                elif current_section == "hvac_system":
                    if "hvac_system" in key:
                        if current_data:
                            energy_data["building_energy_system"]["hvac_system"].append(current_data)
                        current_data = {}
                    current_data[key] = value

                elif current_section == "renewable_energy_sources":
                    if "renewable_energy_sources" in key:
                        if current_data:
                            energy_data["building_energy_system"]["renewable_energy_sources"].append(current_data)
                        current_data = {}
                    current_data[key] = value

    # Append the last set of data for each section
    if current_section == "heat_pump" and current_data:
        energy_data["building_energy_system"]["heat_pump"].append(current_data)
    elif current_section == "heating_system" and current_data:
        energy_data["building_energy_system"]["heating_system"]["types"].append(current_data)
    elif current_section == "lighting_system" and current_data:
        energy_data["building_energy_system"]["lighting_system"]["types"].append(current_data)
    elif current_section == "hvac_system" and current_data:
        energy_data["building_energy_system"]["hvac_system"].append(current_data)
    elif current_section == "renewable_energy_sources" and current_data:
        energy_data["building_energy_system"]["renewable_energy_sources"].append(current_data)

    return energy_data


def save_to_json(data, output_json_path):

    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)
        
    print(f"Energy-related data successfully exported to {output_json_path}")


In [59]:
if __name__ == "__main__":

    energy_file_path = open_energy_txt_file()
    
    if energy_file_path:
        output_json_energy_path = Path.cwd() / 'IDL_files' / 'IDL_energy_types.json'
        output_json_energy_path.parent.mkdir(exist_ok=True)  # Ensure output directory exists

        # Parse and save data
        parsed_data = parse_energy_system(energy_file_path)
        save_to_json(parsed_data, output_json_energy_path)
    else:
        print("No file selected. Exiting.")


Energy-related data successfully exported to c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\IDL_energy_types.json
